## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-16-16-37-36 +0000,nypost,Michigan man fatally shoots teen who broke int...,https://nypost.com/2025/11/16/us-news/dayton-k...
1,2025-11-16-16-36-00 +0000,nypost,"Trump gives update on when $2,000 tariff divid...",https://nypost.com/2025/11/16/us-news/trump-gi...
2,2025-11-16-16-15-28 +0000,nypost,Jets cornerback Kris Boyd in critical conditio...,https://nypost.com/2025/11/16/sports/jets-corn...
3,2025-11-16-16-12-26 +0000,nyt,Arctic Blast Brings Risk of Snow and Ice to Br...,https://www.nytimes.com/2025/11/16/weather/uk-...
4,2025-11-16-15-50-56 +0000,nyt,Why Hollywood’s Star-Studded Movies Are Failin...,https://www.nytimes.com/2025/11/14/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,19
198,dead,9
200,year,8
116,epstein,6
413,president,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
117,2025-11-15-19-24-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...,63
12,2025-11-16-14-50-10 +0000,nypost,Marjorie Taylor Greene insists she still suppo...,https://nypost.com/2025/11/16/us-news/marjorie...,49
52,2025-11-16-08-12-45 +0000,bbc,Feud erupts between Trump and ally Marjorie Ta...,https://www.bbc.com/news/articles/c9wv4dx05q5o...,44
142,2025-11-15-17-07-31 +0000,nyt,Donald Trump Ends Support for Marjorie Taylor ...,https://www.nytimes.com/2025/11/14/us/politics...,40
82,2025-11-16-02-00-00 +0000,wsj,President Trump and his advisers are rushing t...,https://www.wsj.com/economy/consumers/trump-co...,40


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
117,63,2025-11-15-19-24-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...
33,36,2025-11-16-10-39-23 +0000,nypost,Canadian dad dies trying to rescue 5-year-old ...,https://nypost.com/2025/11/16/us-news/canadian...
67,33,2025-11-16-04-19-06 +0000,nypost,Four dead as migrant vessel overturns in rough...,https://nypost.com/2025/11/15/us-news/four-dea...
132,28,2025-11-15-17-54-19 +0000,wapo,Former Fed governor violated central bank’s tr...,https://www.washingtonpost.com/business/2025/1...
87,27,2025-11-16-01-42-19 +0000,cbc,Anti-government protesters clash with riot pol...,https://www.cbc.ca/news/world/gen-z-protest-me...
130,24,2025-11-15-18-15-54 +0000,nypost,Zelensky vows to overhaul Ukraine’s energy sec...,https://nypost.com/2025/11/15/world-news/zelen...
77,21,2025-11-16-02-46-57 +0000,nypost,Married teacher who sexually abused teen— then...,https://nypost.com/2025/11/15/us-news/married-...
122,20,2025-11-15-19-00-00 +0000,nypost,"Terrible customers, low wages and no time for ...",https://nypost.com/2025/11/15/world-news/what-...
71,19,2025-11-16-03-51-02 +0000,missionlocal,‘Mole to Die For’: Mission District chefs cook...,https://missionlocal.org/2025/11/mole-to-die-f...
78,19,2025-11-16-02-29-41 +0000,cbc,"Alice Wong, U.S. disability rights activist an...",https://www.cbc.ca/news/world/alice-wong-obit-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
